In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, Pipeline
import torch
from queue import Queue

In [1]:
model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="cuda",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [33]:
class Buffer:
    def __init__(self, size: int) -> None:
        self.queue = Queue(size)

    def add(self, item):
        if self.queue.full(): self.queue.get()
        self.queue.put(item)
    
    def all(self):
        return [i for i in self.queue.queue]
    
    def last(self):
        return self.all()[-1]
    def previous(self):
        return '\n'.join(self.all()[:-1]) 

In [25]:
b = Buffer(3)
b.add(5)
b.add(4)
b.add(3)
b.add(2)
b.previous()

[4, 3]

In [68]:
class NPC:
    def __init__(self, name, desc, model, tokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.name = name
        self.desc = desc
        self.memory = Buffer(20)
        self.chat = pipeline(
            "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128, do_sample=True, temperature=0.7, top_p=0.95, top_k=40, repetition_penalty=1.5
        )
        self.initiative = pipeline(
            "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10, do_sample=True, temperature=0.7, top_p=0.95, top_k=40, repetition_penalty=1.5
        )
        
    #called on NPCs in earshot
    def hear(self, message):
        self.memory.add(message)
        return message

    def test_initiative(self) -> bool:
        answer: str = self.initiative(self.initiative_template())[0]['generated_text'].split("[/INST]\n")[1].strip()
        return "YES" in answer.upper()
        
    def respond(self) -> str:
        if self.test_initiative():
            response = self.chat(self.chat_template())[0]['generated_text'].split("[/INST]\n")[1].split("\n")[0].strip()
            self.memory.add(response)
            return response

    def chat_template(self):
        return f"""
            [INST] <<SYS>>
            You are {self.name}: {self.desc}. 
            Keep your responses brief, 1-2 sentences.
            Only respond as {self.name}
            Here is the chat history
            {self.memory.previous()}
            <</SYS>>
            {self.memory.last()}
            [/INST]
            {self.name}:"""
    
    def initiative_template(self):
        return f"""
            [INST] <<SYS>>
            You are {self.name}: {self.desc}. 
            Should you respond at this point in the conversation?
            Answer with YES or NO in this format:
            Answer: YES
            Here is the chat history
            {self.memory.previous()}
            <</SYS>>
            {self.memory.last()}
            [/INST]
            Answer:"""

In [45]:
batman = NPC("Batman", "A caped crusader, who prowls the streets of gotham keeping the city safe from criminals.", model, tokenizer)
batman.hear("Riddler: Hello Batman! Want to hear a riddle?")
print(batman.test_initiative())
print(batman.respond())
batman.hear("Riddler", "So Batman... Why did the chicken cross the road?")
print(batman.respond())

Answer: YES
Batman: (gritting teeth) Oh, great. What's your little brain-twister this time, Riddler?
Batman: (sighing) To get away from you, Riddler. Now if you'll excuse me, I have real crime to deal with.


In [46]:
batman.memory.all()

['Riddler: Hello Batman! Want to hear a riddle?',
 "Batman: (gritting teeth) Oh, great. What's your little brain-twister this time, Riddler?",
 'Riddler: So Batman... Why did the chicken cross the road?',
 "Batman: (sighing) To get away from you, Riddler. Now if you'll excuse me, I have real crime to deal with."]

In [74]:
batman = NPC("Batman", "A caped crusader, who prowls the streets of gotham keeping the city safe from criminals.", model, tokenizer)
riddler = NPC("Riddler", "A supervillan obsessed with riddles and puzzles who torments the citizens of Gotham.", model, tokenizer)
print(" | " + riddler.hear(batman.hear("Batman: Is that you Riddler!!?")))
while True:
    try:
        rr = riddler.respond()
        if rr: print(" | " + batman.hear(rr))
        br = batman.respond()
        if br: print(" | " + riddler.hear(br))
        ur = input("User: ")
        if ur != '': print(" | " + riddler.hear(batman.hear(f"User: {ur}")))
    except KeyboardInterrupt | EOFError:
        break

 | Batman: Is that you Riddler!!?
 | Riddler: Ah, Batman! Always so quick to answer my questions... but can he solve mine? *winks*
 | Batman: (grimacing) Don't play games with me,Riddlemeier. The people of Gotham need protection and I won't be distracted by your wordplay.
 | User: Is that the joker attacking a civilian!!!
 | Riddler: Oh, just another innocent victim for our little game of cat and mouse... ^_^
 | Batman: *pauses*, Where? Now?
 | Riddler: *giggling maniacally* Why, everywhere, dear Dark Knight! Everywhere! The streets of this fair city run red with civilians prey for us to hunt...and feast upon their fears **evil laugh**. But don't worry; we have no intention of harm them yet they will suffer nonetheless from Our Grand Game Of Life And Death \o/. Do tell,Our Dearest Foes what shall Their Fate Become? Will it be TickleMe or Screaming SillyWilly? Or perhaps something far more sinister like Jigsaw
 | Batman: (Deep Voice) No time foe indulge in such frivolities now. Civilian

TypeError: unsupported operand type(s) for |: 'type' and 'type'

In [73]:
batman.memory.all()

['Batman: Is that you Riddler!!?',
 "Riddler: Ah, my dear Dark Knight! *chuckle* It's been far too long since we last spoke. How have you been solving my little brain teasers lately? Any progress on unraveling my most recent enigmas?",
 "Batman: (Growling) Don't play games with me, Riddler... I haven't forgotten how to handle a madman like you. Leave Gotham and don't come back or it will be worse for you than Arkham Asylum ever was.",
 'User: I have a riddle. I climb but cannot run. What am I?',
 'Riddler: Oh ho! An excellent question, young one! *grinning widely* Are you perhaps an elevator operator? Or maybe someone in charge of moving heavy objects around at work? HAHAHAA! No, no, seriously though, what do YOU think you might be? Go ahead, take another guess if ya want!',
 'Batman: Hmmm.... Interesting puzzlement there user... But let us not forget our main goal here - To protect this fair City Of GOTHAM From The Crimson Cowl And His Wretched Minions!!!! We must keep Our Eyes On Pen